<a href="https://colab.research.google.com/github/AksleyRios/DataAquosmic/blob/main/PruebaPipeline/Prueba_de_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports/preparación

In [1]:
!pip install openpyxl --upgrade
!pip install utm
import pandas as pd
import numpy as np 
from datetime import datetime
import ee
import utm
import pickle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for utm: filename=utm-0.7.0-py3-none-any.whl size=6108 sha256=8a9ec3a915dfc7d7699c445c0290bcf3d2e7571613972106f91a65bdc2e779fe
  Stored in directory: /root/.cache/pip/wheels/a5/b0/12/7ee4fdb0f9fbb4157100bd02390436ed5d58ebfd3c6d6a0886
Successfully built utm


In [2]:
!earthengine authenticate

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=foD2bFLO5L4PwjRfYC-x1mevdKEHbgWAztXL4gFby3M&tc=bt4nVZUSkd_TbJXWZDKBDxBl1ODLl5JCylWWlc0BQro&cc=GbXsK5IaWuRJThXQkF7jBDXkKaMCcYd2rFxIIyprxgw

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWgGOZ4ZSyi7Qh7Cxxtyvi6QO-WXnSRIW19301RnFDDf-thqdMULn24

Successfully saved authorization token.


In [3]:
ee.Initialize()

#Recopilación de datos

##Longitud, Latitud y Fecha

In [8]:
lonarr=[]
latarr=[]
datearr=[]
print("¿Cuántos puntos tienes?")
n=input()
for i in range(0,int(n)):
  print("Longitud:")
  lon=input()
  lonarr.append(float(lon))
  print("Latitud:")
  lat=input()
  latarr.append(float(lat))
  print("Día:")
  day=input()
  print("Mes:")
  month=input()
  print("Año (YYYY):")
  year=input()
  date=year+'-'+month+'-'+day
  print(date)
  date=datetime.strptime(date, '%Y-%m-%d')
  datearr.append(date)
lon_pd = pd.DataFrame(lonarr)
lon_pd.columns = ['Lon']
lat_pd = pd.DataFrame(latarr)
lat_pd.columns = ['Lat']
date_pd = pd.DataFrame(datearr)
date_pd.columns = ['Date']
Base=lon_pd.join(lat_pd)
Base=Base.join(date_pd)
Base

¿Cuántos puntos tienes?
2
Longitud:
-83.86
Latitud:
43.89
Día:
7
Mes:
12
Año (YYYY):
2017
2017-12-7
Longitud:
-83.64
Latitud:
43.73
Día:
7
Mes:
12
Año (YYYY):
2017
2017-12-7


,Lon,Lat,Date
0,-83.86,43.89,2017-12-07
1,-83.64,43.73,2017-12-07


##Bandas

In [9]:
date_plus = []
data_1f = Base.reset_index()  # make sure indexes pair with number of rows


for index, row in data_1f.iterrows():
    date_row = row['Date']
    date_plus.append(date_row)


date_pd = pd.DataFrame(date_plus)
date_pd.columns = ['Date_ref']

data_1j = Base.join(date_pd)

data_1j['Date_plus'] = data_1j['Date_ref'] + pd.DateOffset(months=1)
data_1j['Date_less'] = data_1j['Date_ref'] + pd.DateOffset(months=-1)

data_1j.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Lon        2 non-null      float64       
 1   Lat        2 non-null      float64       
 2   Date       2 non-null      datetime64[ns]
 3   Date_ref   2 non-null      datetime64[ns]
 4   Date_plus  2 non-null      datetime64[ns]
 5   Date_less  2 non-null      datetime64[ns]
dtypes: datetime64[ns](4), float64(2)
memory usage: 224.0 bytes


In [10]:
B1 = []
B2 = []
B3 = []
B4 = []
B5 = []
B6 = []
B7 = []
B8 = []
B9 = []
B10 = []
B11 = []
ActDate = []

#https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1
landsat = ee.ImageCollection('LANDSAT/LC08/C02/T1')
for index, row in data_1j.iterrows():
  try:
    landsat_1f = landsat.filterDate(row['Date_less'],row['Date_plus'])
    Image_a = ee.Image(landsat_1f.filterBounds(ee.Geometry.Point([row['Lon'], row['Lat']])).sort('CLOUD_COVER').first())
    p = ee.Geometry.Point(row['Lon'], row['Lat'])
    meanDictionary = Image_a.reduceRegion(reducer=ee.Reducer.mean(), geometry=p, scale=1)
    u = meanDictionary.getInfo()
    B1.append(u['B1'])
    B2.append(u['B2'])
    B3.append(u['B3'])
    B4.append(u['B4'])
    B5.append(u['B5'])
    B6.append(u['B6'])
    B7.append(u['B7'])
    B8.append(u['B8'])
    B9.append(u['B9'])
    B10.append(u['B10'])
    B11.append(u['B11'])
    date1 = ee.Date(landsat_1f.filterBounds(ee.Geometry.Point([row['Lon'], row['Lat']])).sort('CLOUD_COVER').first().get('system:time_start'))
    date2 = date1.format('Y-MM-dd').getInfo()
    ActDate.append(date2)
    print(index)
  except:
    B1.append('')
    B2.append('')
    B3.append('')
    B4.append('')
    B5.append('')
    B6.append('')
    B7.append('')
    B8.append('')
    B9.append('')
    B10.append('')
    B11.append('')
    ActDate.append('')

b1_pd = pd.DataFrame(B1)
b1_pd.columns = ['B1']
b2_pd = pd.DataFrame(B2)
b2_pd.columns = ['B2']
b3_pd = pd.DataFrame(B3)
b3_pd.columns = ['B3']
b4_pd = pd.DataFrame(B4)
b4_pd.columns = ['B4']
b5_pd = pd.DataFrame(B5)
b5_pd.columns = ['B5']
b6_pd = pd.DataFrame(B6)
b6_pd.columns = ['B6']
b7_pd = pd.DataFrame(B7)
b7_pd.columns = ['B7']
b8_pd = pd.DataFrame(B8)
b8_pd.columns = ['B8']
b9_pd = pd.DataFrame(B9)
b9_pd.columns = ['B9']
b10_pd = pd.DataFrame(B10)
b10_pd.columns = ['B10']
b11_pd = pd.DataFrame(B11)
b11_pd.columns = ['B11']
actdate_pd = pd.DataFrame(ActDate)
actdate_pd.columns = ['Actual_Date']

data_2j = data_1j.join(actdate_pd)
data_2j = data_2j.join(b1_pd)
data_2j = data_2j.join(b2_pd)
data_2j = data_2j.join(b3_pd)
data_2j = data_2j.join(b4_pd)
data_2j = data_2j.join(b5_pd)
data_2j = data_2j.join(b6_pd)
data_2j = data_2j.join(b7_pd)
data_2j = data_2j.join(b8_pd)
data_2j = data_2j.join(b9_pd)
data_2j = data_2j.join(b10_pd)
data_2j = data_2j.join(b11_pd)

BaseLimpiaBandas=data_2j
BaseLimpiaBandas

0
1


,Lon,Lat,Date,Date_ref,Date_plus,Date_less,Actual_Date,B1,B2,B3,B4,B5,B6,B7,B8,B9,B10,B11
0,-83.86,43.89,2017-12-07,2017-12-07,2018-01-07,2017-11-07,2017-11-29,7944,7438,6625,5838,5277,5054,5020,6288,5011,18980,18179
1,-83.64,43.73,2017-12-07,2017-12-07,2018-01-07,2017-11-07,2017-11-29,7944,7465,6810,6031,5302,5044,5028,6453,5023,19067,18199


In [11]:
col=["B1", "B2", "B3", "B4", "B5", "B6", "B7", "B8", "B9", "B10", "B11"]
for i in col:
  BaseLimpiaBandas[i]=BaseLimpiaBandas[i].apply(lambda x: x/65455)
BaseLimpiaBandas

,Lon,Lat,Date,Date_ref,Date_plus,Date_less,Actual_Date,B1,B2,B3,B4,B5,B6,B7,B8,B9,B10,B11
0,-83.86,43.89,2017-12-07,2017-12-07,2018-01-07,2017-11-07,2017-11-29,0.121366,0.113635,0.101215,0.089191,0.080620,0.077213,0.076694,0.096066,0.076556,0.289970,0.277733
1,-83.64,43.73,2017-12-07,2017-12-07,2018-01-07,2017-11-07,2017-11-29,0.121366,0.114048,0.104041,0.092140,0.081002,0.077061,0.076816,0.098587,0.076740,0.291299,0.278038


#Predicciones

##Modelos

In [12]:
lrOUSA=pickle.load(open('LinearOxigenoUSA.sav','rb'))
lrNUSA=pickle.load(open('LinearNitrogenoUSA.sav','rb'))
lrFUSA=pickle.load(open('LinearFosforoUSA.sav','rb'))
lrCOUSA=pickle.load(open('LinearConductividadUSA.sav','rb'))
lrCLUSA=pickle.load(open('LinearClorofilaUSA.sav','rb'))
lrOMex=pickle.load(open('LinearOxigenoMéxicoLent.sav','rb'))
lrNMex=pickle.load(open('LinearNitrogenoMéxicoLent.sav','rb'))
lrCOMex=pickle.load(open('LinearConductividadMéxicoLent.sav','rb'))
lrTMex=pickle.load(open('LinearTurbiedadMéxicoLent.sav','rb'))
lrFMex=pickle.load(open('LinearFosforoMéxicoLent.sav','rb'))

##Tabla

In [17]:
OxiUSAArr=[]
OxiMexArr=[]
NitUSAArr=[]
NitMexArr=[]
CondUSAArr=[]
CondMexArr=[]
FosfUSAArr=[]
FosfMexArr=[]
ClorofUSAArr=[]
TurbMexArr=[]
for i in range(0,int(n)):
  OxiUSAArr.append(lrOUSA.predict(np.array(BaseLimpiaBandas[['B1','B2','B3','B4','B5','B6','B7','B8','B9','B10','B11']].iloc[i]).reshape(1,-1)))
  OxiMexArr.append(lrOMex.predict(np.array(BaseLimpiaBandas[['B2','B3','B4','B5','B6','B8']].iloc[i]).reshape(1,-1)))
  NitUSAArr.append(lrNUSA.predict(np.array(BaseLimpiaBandas[['B1','B2','B3','B4','B5','B6','B7','B8','B9','B10','B11']].iloc[i]).reshape(1,-1)))
  NitMexArr.append(lrNMex.predict(np.array(BaseLimpiaBandas[['B2','B3','B4','B5','B6','B8']].iloc[i]).reshape(1,-1)))
  FosfUSAArr.append(lrFUSA.predict(np.array(BaseLimpiaBandas[['B1','B2','B3','B4','B5','B6','B7','B8','B9','B10','B11']].iloc[i]).reshape(1,-1)))
  FosfMexArr.append(lrFMex.predict(np.array(BaseLimpiaBandas[['B1','B2','B3','B4','B5','B6','B8']].iloc[i]).reshape(1,-1)))
  CondUSAArr.append(lrCOUSA.predict(np.array(BaseLimpiaBandas[['B1','B2','B3','B4','B5','B6','B7','B8','B9','B10','B11']].iloc[i]).reshape(1,-1)))
  CondMexArr.append(lrCOMex.predict(np.array(BaseLimpiaBandas[['B1','B2','B3','B4','B5','B6','B8']].iloc[i]).reshape(1,-1)))
  ClorofUSAArr.append(lrCLUSA.predict(np.array(BaseLimpiaBandas[['B1','B2','B3','B4','B5','B6','B7','B8','B9','B10','B11']].iloc[i]).reshape(1,-1)))
  TurbMexArr.append(lrTMex.predict(np.array(BaseLimpiaBandas[['B1','B2','B3','B4','B5','B6','B8']].iloc[i]).reshape(1,-1)))


OxiUSA_pd = pd.DataFrame(OxiUSAArr)
OxiUSA_pd.columns = ['Oxigeno USA']
OxiMex_pd = pd.DataFrame(OxiMexArr)
OxiMex_pd.columns = ['Oxigeno Mex']
NitUSA_pd = pd.DataFrame(NitUSAArr)
NitUSA_pd.columns = ['Nitrogeno USA']
NitMex_pd = pd.DataFrame(NitMexArr)
NitMex_pd.columns = ['Nitrogeno Mex']
FosfUSA_pd = pd.DataFrame(FosfUSAArr)
FosfUSA_pd.columns = ['Fosforo USA']
FosfMex_pd = pd.DataFrame(FosfMexArr)
FosfMex_pd.columns = ['Fosforo Mex']
CondUSA_pd = pd.DataFrame(CondUSAArr)
CondUSA_pd.columns = ['Conductividad USA']
CondMex_pd = pd.DataFrame(CondMexArr)
CondMex_pd.columns = ['Conductividad Mex']
TurbMex_pd = pd.DataFrame(TurbMexArr)
TurbMex_pd.columns = ['Turbiedad Mex']
ClorofUSA_pd = pd.DataFrame(ClorofUSAArr)
ClorofUSA_pd.columns = ['Clorofila USA']

Tabla=Base.join(OxiUSA_pd)
Tabla=Tabla.join(OxiMex_pd)
Tabla=Tabla.join(NitUSA_pd)
Tabla=Tabla.join(NitMex_pd)
Tabla=Tabla.join(FosfUSA_pd)
Tabla=Tabla.join(FosfMex_pd)
Tabla=Tabla.join(CondUSA_pd)
Tabla=Tabla.join(CondMex_pd)
Tabla=Tabla.join(ClorofUSA_pd)
Tabla=Tabla.join(TurbMex_pd)

Tabla

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  "X does not h

,Lon,Lat,Date,Oxigeno USA,Oxigeno Mex,Nitrogeno USA,Nitrogeno Mex,Fosforo USA,Fosforo Mex,Conductividad USA,Conductividad Mex,Clorofila USA,Turbiedad Mex
0,-83.86,43.89,2017-12-07,10.985928,6.482304,0.063941,1.231696,0.112820,0.326582,4250.128981,543.587507,12.793305,6.156554
1,-83.64,43.73,2017-12-07,10.918079,6.509474,0.061596,1.252367,0.116692,0.340682,4622.586102,548.940759,13.277578,8.454699
